In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
import pandas as pd
import numpy as np
import csv

In [11]:
train_tok = pd.read_csv("train_data_tokenized.csv")
train_tok

,Unnamed: 0,Record Number,Title,Token,Tag,na
0,0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell,False
1,1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke,False
2,3,1,Supreme Nike SB Dunk High By any Means Red US1...,SB Dunk,Produktlinie,True
3,4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ,False
4,7,1,Supreme Nike SB Dunk High By any Means Red US1...,By any Means,Modell,True
...,...,...,...,...,...,...
44750,55177,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sneaker,Stil,False
44751,55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe Wanderschuh,Produktart,True
44752,55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag,False
44753,55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag,False


In [12]:
try_ebay_text = train_tok.Title[0]

doc = nlp(try_ebay_text)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [8]:
text = "What video sharing service did Steve Chen, Chad Hurley, and Jawed Karim create in 2005?"
 


In [21]:
tok_tags = train_tok.loc[train_tok.Tag != "No Tag", ["Token", "Tag"]]
tok_tags

,Token,Tag
0,Supreme,Modell
1,Nike,Marke
2,SB Dunk,Produktlinie
3,High,Schuhschaft-Typ
4,By any Means,Modell
...,...,...
44747,Trekking,Aktivität
44748,Schuhe,Produktart
44749,Outdoor,Anlass
44750,Sneaker,Stil


In [36]:
tok_tags = tok_tags.assign(START=0)
tok_tags["END"] = tok_tags.apply(lambda x: len(x.Token), axis=1)
tok_tags["as_arr"] = tok_tags.loc[:, ["START", "END", "Tag"]].values.tolist()
tok_arr = tok_tags.loc[:, ["Token", "as_arr"]]
tok_arr

,Token,as_arr
0,Supreme,"[0, 7, Modell]"
1,Nike,"[0, 4, Marke]"
2,SB Dunk,"[0, 7, Produktlinie]"
3,High,"[0, 4, Schuhschaft-Typ]"
4,By any Means,"[0, 12, Modell]"
...,...,...
44747,Trekking,"[0, 8, Aktivität]"
44748,Schuhe,"[0, 6, Produktart]"
44749,Outdoor,"[0, 7, Anlass]"
44750,Sneaker,"[0, 7, Stil]"


In [39]:
spacy_training_data = list(tok_arr.itertuples(index=False, name=None))
spacy_training_data

[('Supreme', [0, 7, 'Modell']),
 ('Nike', [0, 4, 'Marke']),
 ('SB Dunk', [0, 7, 'Produktlinie']),
 ('High', [0, 4, 'Schuhschaft-Typ']),
 ('By any Means', [0, 12, 'Modell']),
 ('Red', [0, 3, 'Farbe']),
 ('US10', [0, 4, 'US-Schuhgröße']),
 ('EU44', [0, 4, 'EU-Schuhgröße']),
 ('Logo', [0, 4, 'Akzente']),
 ('Air Force', [0, 9, 'Produktlinie']),
 ('New Balance', [0, 11, 'Marke']),
 ('530', [0, 3, 'Modell']),
 ('Männer', [0, 6, 'Abteilung']),
 ('Frauen', [0, 6, 'Abteilung']),
 ('Laufschuhe', [0, 10, 'Produktart']),
 ('Buchstaben N', [0, 12, 'Muster']),
 ('Laufschuhe', [0, 10, 'Produktart']),
 ('MICHAEL KORS', [0, 12, 'Marke']),
 ('Sneaker', [0, 7, 'Stil']),
 ('39', [0, 2, 'EU-Schuhgröße']),
 ('New Balance', [0, 11, 'Marke']),
 ('ML 574 EGO', [0, 10, 'Modell']),
 ('Turnschuhe', [0, 10, 'Produktart']),
 ('grün', [0, 4, 'Farbe']),
 ('Nike', [0, 4, 'Marke']),
 ('air jordan', [0, 10, 'Produktlinie']),
 ('9 og', [0, 4, 'Modell']),
 ('space', [0, 5, 'Farbe']),
 ('jam', [0, 3, 'Modell']),
 ('hare', 

In [48]:
db = spacy.tokens.DocBin()
for text, annotations in spacy_training_data:
    doc = nlp(text)
    ents = []
    start, end, label = annotations
    span = doc.char_span(start, end, label=label)
    ents.append(span)
    doc.ents = ents
    db.add(doc)
db.to_disk("./indiv_docs.spacy")

In [54]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")
nlp.add_pipe("parser")
nlp.add_pipe("tok2vec")
nlp.add_pipe("tagger")
# This is a problem because it needs entities and sentence boundaries
nlp.add_pipe("ner")
nlp.add_pipe("entity_linker")
analysis = nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component       Assigns               Requires         Scores             Retokenizes
-   -------------   -------------------   --------------   ----------------   -----------
0   sentencizer     token.is_sent_start                    sents_f            False      
                    doc.sents                              sents_p                       
                                                           sents_r                       
                                                                                         
1   parser          token.dep                              dep_uas            False      
                    token.head                             dep_las                       
                    token.is_sent_start                    dep_las_per_type              
                    doc.sents                              sents_p                       
                    

In [58]:
in_sample = pd.unique(train_df.Title)


'Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force'

In [79]:
nlp_ner = spacy.load('model-last')
doc = nlp_ner(in_sample[0][7:13])
spacy.displacy.render(doc, style="ent", jupyter=True)

Gets particular words right.

In [80]:
nlp_ner = spacy.load('model-last')
doc = nlp_ner(in_sample[0])
spacy.displacy.render(doc, style="ent", jupyter=True)

Cannot get sentences as a whole correct.